In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import gc

# ===================================================================
# 1. Load and Prepare Data
# ===================================================================
print("Loading data...")
df = pd.read_csv("/content/normalized_features.csv")

df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df = df.sort_values(by=['Location', 'Timestamp']).reset_index(drop=True)

# ==> CHANGE 1: Apply log transformation to the target variable 'out'
# We create a new column for the transformed target to keep the original for validation.
df['out_transformed'] = np.log1p(df['out'])

le = LabelEncoder()
df['Location_encoded'] = le.fit_transform(df['Location'])


# ===================================================================
# 2. Feature Engineering for Time-Series
# ===================================================================
print("Engineering features...")

def create_time_series_features(df_in):
    df_out = df_in.copy()
    features_to_lag = ['out', 't2m', 'gust', 'cape', 'prate']
    lag_periods = [1, 2, 3, 24, 48]

    for feature in features_to_lag:
        for lag in lag_periods:
            df_out[f'{feature}_lag_{lag}'] = df_out.groupby('Location')[feature].shift(lag)

    df_out['gust_roll_max_3h'] = df_out.groupby('Location')['gust'].transform(
        lambda x: x.shift(1).rolling(window=3).max()
    )
    df_out['hour'] = df_out['Timestamp'].dt.hour
    df_out['dayofweek'] = df_out['Timestamp'].dt.dayofweek
    return df_out

df_features = create_time_series_features(df)

features = [col for col in df_features.columns if col not in [
    'out', 'out_transformed', 'Timestamp', 'Location'
]]


# ===================================================================
# 3. Create a Time-Based Validation Split
# ===================================================================
print("Creating time-based train/validation split...")
validation_start_date = df_features['Timestamp'].max() - pd.Timedelta(days=7)

train_df = df_features[df_features['Timestamp'] < validation_start_date].copy()
val_df = df_features[df_features['Timestamp'] >= validation_start_date].copy()

train_df.dropna(inplace=True)


# ===================================================================
# 4. Train Models with Direct Forecasting Strategy
# ===================================================================
print("Starting model training...")
MAX_HORIZON = 48
models = {}

for h in range(1, MAX_HORIZON + 1):
    print(f"\n===== Training model for horizon t+{h} =====")

    # Use the TRANSFORMED 'out' column as the target
    y_train_h = train_df.groupby('Location')['out_transformed'].shift(-h)
    X_train_h = train_df[features]

    valid_indices = ~y_train_h.isna()
    X_train_h, y_train_h = X_train_h[valid_indices], y_train_h[valid_indices]

    # Create validation set for this horizon
    X_val_h = val_df[features]
    y_val_h = val_df.groupby('Location')['out_transformed'].shift(-h).dropna()
    X_val_h = X_val_h.loc[y_val_h.index]

    # ==> CHANGE 2: Objective is 'regression_l2' (RMSE)
    lgbm = lgb.LGBMRegressor(
        objective='regression_l2',
        n_estimators=2000,
        learning_rate=0.01,
        num_leaves=40,
        random_state=42,
        n_jobs=-1,
        colsample_bytree=0.7,
        subsample=0.7,
        device='gpu'
    )

    lgbm.fit(
        X_train_h, y_train_h,
        eval_set=[(X_val_h, y_val_h)],
        eval_metric='rmse',
        # ==> CHANGE 3: Callback will now print the loss as it improves
        callbacks=[lgb.early_stopping(100, verbose=True)]
    )

    models[h] = lgbm
    gc.collect()

print("\nModel training complete.")


# ===================================================================
# 5. Generate and Evaluate Predictions
# ===================================================================
print("Generating and evaluating predictions on the validation set...")

validation_predictions = []
for loc_id in val_df['Location'].unique():
    loc_df = val_df[val_df['Location'] == loc_id].copy()

    # Use the features from the first timestamp of the validation set for this location
    X_pred_loc = loc_df[features].iloc[[0]]

    for h in range(1, MAX_HORIZON + 1):
        if h > len(loc_df): continue # Cannot validate if horizon is longer than remaining data

        prediction_transformed = models[h].predict(X_pred_loc)[0]

        # ==> CHANGE 4: Reverse the transformation using expm1
        prediction_original_scale = np.expm1(prediction_transformed)
        prediction_original_scale = max(0, prediction_original_scale)

        # Get the true value from the original 'out' column
        true_value = loc_df['out'].iloc[h-1]

        validation_predictions.append({
            'pred': prediction_original_scale,
            'true': true_value
        })

final_preds_df = pd.DataFrame(validation_predictions)
rmse = np.sqrt(mean_squared_error(final_preds_df['true'], final_preds_df['pred']))
print(f"\n✅ Final Validation RMSE on original scale: {rmse:.4f}")

Loading data...
Engineering features...
Creating time-based train/validation split...
Starting model training...

===== Training model for horizon t+1 =====
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 24943
[LightGBM] [Info] Number of data points in the train set: 161269, number of used features: 116
[LightGBM] [Info] Using GPU Device: NVIDIA A100-SXM4-40GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 81 dense feature groups (12.92 MB) transferred to GPU in 0.012061 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 0.673712
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[691]	valid_0's rmse: 1.10076	valid_0's l2: 1.21166

===== Training model for horizon t+2 =====
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total 

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import gc
import joblib

# ===================================================================
# 1. Load and Prepare Data
# ===================================================================
print("Loading data...")
df = pd.read_csv("/content/normalized_features.csv")
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df = df.sort_values(by=['Location', 'Timestamp']).reset_index(drop=True)
df['out_transformed'] = np.log1p(df['out'])
le = LabelEncoder()
df['Location_encoded'] = le.fit_transform(df['Location'])

# ===================================================================
# 2. Feature Engineering
# ===================================================================
print("Engineering time series features...")
def create_time_series_features(df_in):
    df_out = df_in.copy()
    # Only create lags for features in your selected set
    features_to_lag = [f for f in [
        'out', 't2m', 'gust', 'cape', 'cape_1'
    ] if f in df_out.columns]
    lag_periods = [1, 2, 3, 6, 8, 10, 12, 15, 24, 48]
    for feature in features_to_lag:
        for lag in lag_periods:
            df_out[f'{feature}_lag_{lag}'] = df_out.groupby('Location')[feature].shift(lag)

    # Only compute rolling/std for features that exist
    if 'gust' in df_out.columns:
        df_out['gust_roll_max_3h'] = df_out.groupby('Location')['gust'].transform(lambda x: x.shift(1).rolling(3).max())
        df_out['gust_std_6h'] = df_out.groupby('Location')['gust'].transform(lambda x: x.shift(1).rolling(6).std())
        df_out['gust_acceleration_3h'] = df_out['gust_lag_1'] - df_out['gust_lag_3']

    if 't2m' in df_out.columns:
        df_out['t2m_std_12h'] = df_out.groupby('Location')['t2m'].transform(lambda x: x.shift(1).rolling(12).std())

    if 'cape' in df_out.columns and 'cape_lag_1' in df_out.columns and 'cape_lag_2' in df_out.columns:
        df_out['cape_change_1h'] = df_out['cape_lag_1'] - df_out['cape_lag_2']

    df_out['hour'] = df_out['Timestamp'].dt.hour
    df_out['dayofweek'] = df_out['Timestamp'].dt.dayofweek
    return df_out

df_features = create_time_series_features(df)

# ===================================================================
# 3. Define Final Feature List
# ===================================================================
features = [
    'fricv', 'cnwat', 'max_10si', 'sdlwrf', 'slhtf', 'mslma','pt','gh_4','t', 't2m',
    'sp', 'sulwrf','gust', 'cape', 'blh', 'sh2', 'd2m', 'lftx',  'cape_1', 'pwat'
]
print(f"Training will use your selected set of {len(features)} unique features.")

# ===================================================================
# 4. Create a Time-Based Validation Split
# ===================================================================
print("Creating time-based train/validation split...")
validation_start_date = df_features['Timestamp'].max() - pd.Timedelta(days=7)
train_df = df_features[df_features['Timestamp'] < validation_start_date].copy()
val_df = df_features[df_features['Timestamp'] >= validation_start_date].copy()
train_df.dropna(subset=features, inplace=True)

# ===================================================================
# 5. Train Models
# ===================================================================
print("Starting model training with the selected features...")

best_params = {
    'objective': 'regression_l2', 'metric': 'rmse', 'n_estimators': 3500,
    'learning_rate': 0.013163, 'num_leaves': 98, 'max_depth': 18,
    'subsample': 0.78, 'colsample_bytree': 0.65, 'reg_alpha': 0.45,
    'reg_lambda': 0.55, 'random_state': 42, 'n_jobs': -1,
}

MAX_HORIZON = 48
models = {}
for h in range(1, MAX_HORIZON + 1):
    print(f"\n===== Training model for horizon t+{h} =====")
    y_train_h = train_df.groupby('Location')['out_transformed'].shift(-h)
    X_train_h = train_df[features]
    valid_indices = ~y_train_h.isna()
    X_train_h, y_train_h = X_train_h[valid_indices], y_train_h[valid_indices]
    X_val_h = val_df[features]
    y_val_h = val_df.groupby('Location')['out_transformed'].shift(-h).dropna()
    X_val_h = X_val_h.loc[y_val_h.index]
    lgbm = lgb.LGBMRegressor(**best_params)
    lgbm.fit(
        X_train_h, y_train_h, eval_set=[(X_val_h, y_val_h)], eval_metric='rmse',
        callbacks=[lgb.early_stopping(100, verbose=True)]
    )
    models[h] = lgbm
    gc.collect()

print("\nModel training complete.")
joblib.dump(models, 'lgbm_selected_features_models.joblib')
print("✅ Selected-feature models saved to lgbm_selected_features_models.joblib")

# ===================================================================
# 6. Evaluate Predictions
# ===================================================================
print("Generating and evaluating predictions on the validation set...")
validation_predictions = []
for loc_id in val_df['Location'].unique():
    loc_df = val_df[val_df['Location'] == loc_id].copy()
    X_pred_loc = loc_df[features].iloc[[0]]
    for h in range(1, MAX_HORIZON + 1):
        if h > len(loc_df): continue
        prediction_transformed = models[h].predict(X_pred_loc)[0]
        prediction_original_scale = np.expm1(prediction_transformed)
        prediction_original_scale = max(0, prediction_original_scale)
        true_value = loc_df['out'].iloc[h-1]
        validation_predictions.append({'pred': prediction_original_scale, 'true': true_value})

final_preds_df = pd.DataFrame(validation_predictions)
rmse = np.sqrt(mean_squared_error(final_preds_df['true'], final_preds_df['pred']))
print(f"\n✅ Final Validation RMSE with selected features: {rmse:.4f}")


Loading data...
Engineering time series features...
Training will use your selected set of 20 unique features.
Creating time-based train/validation split...
Starting model training with the selected features...

===== Training model for horizon t+1 =====
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006734 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4769
[LightGBM] [Info] Number of data points in the train set: 165253, number of used features: 20
[LightGBM] [Info] Start training from score 0.692920
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[819]	valid_0's rmse: 1.79206

===== Training model for horizon t+2 =====
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006311 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4769
[LightGBM] [Info] Number of data po

second try

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import gc
import joblib

# ===================================================================
# 1. Load and Prepare Data
# ===================================================================
print("Loading data...")
df = pd.read_csv("/content/normalized_features.csv")
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df = df.sort_values(by=['Location', 'Timestamp']).reset_index(drop=True)
df['out_transformed'] = np.log1p(df['out'])
le = LabelEncoder()
df['Location_encoded'] = le.fit_transform(df['Location'])

# ===================================================================
# 2. Feature Engineering
# ===================================================================
print("Engineering time series features...")
def create_time_series_features(df_in):
    df_out = df_in.copy()
    # Only create lags for features in your selected set
    features_to_lag = [f for f in [
        'out', 't2m', 'gust', 'cape', 'cape_1'
    ] if f in df_out.columns]
    lag_periods = [1, 2, 3, 6, 8, 10, 12, 15, 24, 48]
    for feature in features_to_lag:
        for lag in lag_periods:
            df_out[f'{feature}_lag_{lag}'] = df_out.groupby('Location')[feature].shift(lag)

    # Only compute rolling/std for features that exist
    if 'gust' in df_out.columns:
        df_out['gust_roll_max_3h'] = df_out.groupby('Location')['gust'].transform(lambda x: x.shift(1).rolling(3).max())
        df_out['gust_std_6h'] = df_out.groupby('Location')['gust'].transform(lambda x: x.shift(1).rolling(6).std())
        df_out['gust_acceleration_3h'] = df_out['gust_lag_1'] - df_out['gust_lag_3']

    if 't2m' in df_out.columns:
        df_out['t2m_std_12h'] = df_out.groupby('Location')['t2m'].transform(lambda x: x.shift(1).rolling(12).std())

    if 'cape' in df_out.columns and 'cape_lag_1' in df_out.columns and 'cape_lag_2' in df_out.columns:
        df_out['cape_change_1h'] = df_out['cape_lag_1'] - df_out['cape_lag_2']

    df_out['hour'] = df_out['Timestamp'].dt.hour
    df_out['dayofweek'] = df_out['Timestamp'].dt.dayofweek
    return df_out

df_features = create_time_series_features(df)

# ===================================================================
# 3. Define Final Feature List
# ===================================================================
features = [
    'fricv', 'cnwat', 'max_10si', 'sdlwrf', 'slhtf', 'mslma','pt','gh_4','t', 't2m',
    'sp', 'sulwrf','gust', 'cape', 'blh', 'sh2', 'd2m', 'lftx',  'cape_1', 'pwat'
]
print(f"Training will use your selected set of {len(features)} unique features.")

# ===================================================================
# 4. Create a Time-Based Validation Split
# ===================================================================
print("Creating time-based train/validation split...")
validation_start_date = df_features['Timestamp'].max() - pd.Timedelta(days=7)
train_df = df_features[df_features['Timestamp'] < validation_start_date].copy()
val_df = df_features[df_features['Timestamp'] >= validation_start_date].copy()
train_df.dropna(subset=features, inplace=True)

# ===================================================================
# 5. Train Models
# ===================================================================
print("Starting model training with the selected features...")

best_params = {
    'objective': 'regression_l2', 'metric': 'rmse', 'n_estimators': 3500,
    'learning_rate': 0.013163, 'num_leaves': 98, 'max_depth': 18,
    'subsample': 0.78, 'colsample_bytree': 0.65, 'reg_alpha': 0.45,
    'reg_lambda': 0.55, 'random_state': 42, 'n_jobs': -1,
}

MAX_HORIZON = 48
models = {}
for h in range(1, MAX_HORIZON + 1):
    print(f"\n===== Training model for horizon t+{h} =====")
    y_train_h = train_df.groupby('Location')['out_transformed'].shift(-h)
    X_train_h = train_df[features]
    valid_indices = ~y_train_h.isna()
    X_train_h, y_train_h = X_train_h[valid_indices], y_train_h[valid_indices]
    X_val_h = val_df[features]
    y_val_h = val_df.groupby('Location')['out_transformed'].shift(-h).dropna()
    X_val_h = X_val_h.loc[y_val_h.index]
    lgbm = lgb.LGBMRegressor(**best_params)
    lgbm.fit(
        X_train_h, y_train_h, eval_set=[(X_val_h, y_val_h)], eval_metric='rmse',
        callbacks=[lgb.early_stopping(100, verbose=True)]
    )
    models[h] = lgbm
    gc.collect()

print("\nModel training complete.")
joblib.dump(models, 'lgbm_selected_features_models.joblib')
print("✅ Selected-feature models saved to lgbm_selected_features_models.joblib")

# ===================================================================
# 6. Evaluate Predictions
# ===================================================================
print("Generating and evaluating predictions on the validation set...")
validation_predictions = []
for loc_id in val_df['Location'].unique():
    loc_df = val_df[val_df['Location'] == loc_id].copy()
    X_pred_loc = loc_df[features].iloc[[0]]
    for h in range(1, MAX_HORIZON + 1):
        if h > len(loc_df): continue
        prediction_transformed = models[h].predict(X_pred_loc)[0]
        prediction_original_scale = np.expm1(prediction_transformed)
        prediction_original_scale = max(0, prediction_original_scale)
        true_value = loc_df['out'].iloc[h-1]
        validation_predictions.append({'pred': prediction_original_scale, 'true': true_value})

final_preds_df = pd.DataFrame(validation_predictions)
rmse = np.sqrt(mean_squared_error(final_preds_df['true'], final_preds_df['pred']))
print(f"\n✅ Final Validation RMSE with selected features: {rmse:.4f}")


Loading data...
Engineering time series features...
Training will use your selected set of 20 unique features.
Creating time-based train/validation split...
Starting model training with the selected features...

===== Training model for horizon t+1 =====
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005732 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4769
[LightGBM] [Info] Number of data points in the train set: 165253, number of used features: 20
[LightGBM] [Info] Start training from score 0.692920
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[819]	valid_0's rmse: 1.79206

===== Training model for horizon t+2 =====
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005812 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4769
[LightGBM] [Info] Number of data po

In [ ]:

# ===================================================================
# 4. Generate Predictions for Submission
# ===================================================================
print("Generating predictions for submission files...")

# Get the last row of data for each location from the full feature-engineered dataframe.
# This row contains the most recent lagged features needed for prediction.
last_known_data = df_features.groupby('Location').last().reset_index()

# Load the submission templates to get the required timestamps and locations.
sub_template_24h = pd.read_csv('/content/submission_template_24h.csv')
sub_template_48h = pd.read_csv('/content/submission_template_48h.csv')

sub_template_24h['timestamp'] = pd.to_datetime(sub_template_24h['timestamp'])
sub_template_48h['timestamp'] = pd.to_datetime(sub_template_48h['timestamp'])


all_predictions = []

for loc_encoded, loc_id in enumerate(le.classes_):
    # Get the feature set for the current location
    X_pred_loc = last_known_data[last_known_data['Location'] == loc_id][features]

    # If a location has no data, we can't predict
    if X_pred_loc.empty:
        continue

    for h in range(1, MAX_HORIZON + 1):
        # Predict using the model for horizon 'h'
        prediction = models[h].predict(X_pred_loc)[0]

        # Ensure prediction is non-negative
        prediction = max(0, prediction)

        # The prediction timestamp is h hours after the last known timestamp
        pred_timestamp = last_known_data.loc[last_known_data['Location'] == loc_id, 'Timestamp'].iloc[0] + pd.Timedelta(hours=h)

        all_predictions.append({
            'timestamp': pred_timestamp,
            'location': loc_id,
            'pred': prediction
        })

predictions_df = pd.DataFrame(all_predictions)


# ===================================================================
# 5. Create and Save Submission Files
# ===================================================================
print("Saving submission files...")

# Create 24-hour submission file
submission_24h = sub_template_24h[['timestamp', 'location']].copy()
submission_24h = submission_24h.merge(predictions_df, on=['timestamp', 'location'], how='left')
submission_24h['pred'] = submission_24h['pred'].fillna(0) # Fill any misses with 0

# Create 48-hour submission file
submission_48h = sub_template_48h[['timestamp', 'location']].copy()
submission_48h = submission_48h.merge(predictions_df, on=['timestamp', 'location'], how='left')
submission_48h['pred'] = submission_48h['pred'].fillna(0) # Fill any misses with 0


# Save to CSV in the specified format without the index
submission_24h.to_csv('submission_24h.csv', index=False)
submission_48h.to_csv('submission_48h.csv', index=False)

print(f"✅ Successfully created submission_24h.csv and submission_48h.csv")

Generating predictions for submission files...


/tmp/ipython-input-269123749.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sub_template_24h['timestamp'] = pd.to_datetime(sub_template_24h['timestamp'])
/tmp/ipython-input-269123749.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sub_template_48h['timestamp'] = pd.to_datetime(sub_template_48h['timestamp'])


Saving submission files...
✅ Successfully created submission_24h.csv and submission_48h.csv


In [ ]:
import joblib

joblib.dump(models, 'lgbm_direct_forecasting_models.joblib')
print("✅ Models have been saved to lgbm_direct_forecasting_models.joblib")

✅ Models have been saved to lgbm_direct_forecasting_models.joblib


TRYING TO IMPROVE RMSE EVEN FURTHER

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import gc

# ===================================================================
# 1. Load and Prepare Data
# ===================================================================
print("Loading data...")
df = pd.read_csv("/content/normalized_features.csv")

df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df = df.sort_values(by=['Location', 'Timestamp']).reset_index(drop=True)

# ==> CHANGE 1: Apply log transformation to the target variable 'out'
# We create a new column for the transformed target to keep the original for validation.
df['out_transformed'] = np.log1p(df['out'])

le = LabelEncoder()
df['Location_encoded'] = le.fit_transform(df['Location'])


# ===================================================================
# 2. Feature Engineering for Time-Series
# ===================================================================
print("Engineering features...")

def create_time_series_features(df_in):
    df_out = df_in.copy()
    features_to_lag = ['out', 't2m', 'gust', 'cape', 'prate', 'fricv']
    lag_periods = [1, 2, 3, 24, 48]

    for feature in features_to_lag:
        for lag in lag_periods:
            df_out[f'{feature}_lag_{lag}'] = df_out.groupby('Location')[feature].shift(lag)

    df_out['gust_roll_max_3h'] = df_out.groupby('Location')['gust'].transform(
        lambda x: x.shift(1).rolling(window=3).max()
    )
    df_out['hour'] = df_out['Timestamp'].dt.hour
    df_out['dayofweek'] = df_out['Timestamp'].dt.dayofweek
    return df_out

df_features = create_time_series_features(df)

features = [col for col in df_features.columns if col not in [
    'out', 'out_transformed', 'Timestamp', 'Location'
]]


# ===================================================================
# 3. Create a Time-Based Validation Split
# ===================================================================
print("Creating time-based train/validation split...")
validation_start_date = df_features['Timestamp'].max() - pd.Timedelta(days=7)

train_df = df_features[df_features['Timestamp'] < validation_start_date].copy()
val_df = df_features[df_features['Timestamp'] >= validation_start_date].copy()

train_df.dropna(inplace=True)


# ===================================================================
# 4. Train Models with Direct Forecasting Strategy
# ===================================================================
print("Starting model training...")
MAX_HORIZON = 48
models = {}

for h in range(1, MAX_HORIZON + 1):
    print(f"\n===== Training model for horizon t+{h} =====")

    # Use the TRANSFORMED 'out' column as the target
    y_train_h = train_df.groupby('Location')['out_transformed'].shift(-h)
    X_train_h = train_df[features]

    valid_indices = ~y_train_h.isna()
    X_train_h, y_train_h = X_train_h[valid_indices], y_train_h[valid_indices]

    # Create validation set for this horizon
    X_val_h = val_df[features]
    y_val_h = val_df.groupby('Location')['out_transformed'].shift(-h).dropna()
    X_val_h = X_val_h.loc[y_val_h.index]

    # ==> CHANGE 2: Objective is 'regression_l2' (RMSE)
    lgbm = lgb.LGBMRegressor(
        objective='regression_l2',
        n_estimators=3000,
        learning_rate=0.01,
        num_leaves=60,
        random_state=42,
        n_jobs=-1,
        colsample_bytree=0.7,
        subsample=0.6,
    )

    lgbm.fit(
        X_train_h, y_train_h,
        eval_set=[(X_val_h, y_val_h)],
        eval_metric='rmse',
        # ==> CHANGE 3: Callback will now print the loss as it improves
        callbacks=[lgb.early_stopping(100, verbose=True)]
    )

    models[h] = lgbm
    gc.collect()

print("\nModel training complete.")


# ===================================================================
# 5. Generate and Evaluate Predictions
# ===================================================================
print("Generating and evaluating predictions on the validation set...")

validation_predictions = []
for loc_id in val_df['Location'].unique():
    loc_df = val_df[val_df['Location'] == loc_id].copy()

    # Use the features from the first timestamp of the validation set for this location
    X_pred_loc = loc_df[features].iloc[[0]]

    for h in range(1, MAX_HORIZON + 1):
        if h > len(loc_df): continue # Cannot validate if horizon is longer than remaining data

        prediction_transformed = models[h].predict(X_pred_loc)[0]

        # ==> CHANGE 4: Reverse the transformation using expm1
        prediction_original_scale = np.expm1(prediction_transformed)
        prediction_original_scale = max(0, prediction_original_scale)

        # Get the true value from the original 'out' column
        true_value = loc_df['out'].iloc[h-1]

        validation_predictions.append({
            'pred': prediction_original_scale,
            'true': true_value
        })

final_preds_df = pd.DataFrame(validation_predictions)
rmse = np.sqrt(mean_squared_error(final_preds_df['true'], final_preds_df['pred']))
print(f"\n✅ Final Validation RMSE on original scale: {rmse:.4f}")

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import gc

# ===================================================================
# 1. Load and Prepare Data
# ===================================================================
print("Loading data...")
df = pd.read_csv("/content/train.csv")

df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df = df.sort_values(by=['Location', 'Timestamp']).reset_index(drop=True)

# ==> CHANGE 1: Apply log transformation to the target variable 'out'
# We create a new column for the transformed target to keep the original for validation.
df['out_transformed'] = np.log1p(df['out'])

le = LabelEncoder()
df['Location_encoded'] = le.fit_transform(df['Location'])


# ===================================================================
# 2. Feature Engineering for Time-Series
# ===================================================================
print("Engineering features...")

def create_time_series_features(df_in):
    df_out = df_in.copy()
    features_to_lag = ['out', 't2m', 'gust', 'cape', 'prate']
    lag_periods = [1, 2, 3, 24, 48]

    for feature in features_to_lag:
        for lag in lag_periods:
            df_out[f'{feature}_lag_{lag}'] = df_out.groupby('Location')[feature].shift(lag)

    df_out['gust_roll_max_3h'] = df_out.groupby('Location')['gust'].transform(
        lambda x: x.shift(1).rolling(window=3).max()
    )
    df_out['hour'] = df_out['Timestamp'].dt.hour
    df_out['dayofweek'] = df_out['Timestamp'].dt.dayofweek
    return df_out

df_features = create_time_series_features(df)

features = [col for col in df_features.columns if col not in [
    'out', 'out_transformed', 'Timestamp', 'Location'
]]


# ===================================================================
# 3. Create a Time-Based Validation Split
# ===================================================================
print("Creating time-based train/validation split...")
validation_start_date = df_features['Timestamp'].max() - pd.Timedelta(days=7)

train_df = df_features[df_features['Timestamp'] < validation_start_date].copy()
val_df = df_features[df_features['Timestamp'] >= validation_start_date].copy()

train_df.dropna(inplace=True)


# ===================================================================
# 4. Train Models with Direct Forecasting Strategy
# ===================================================================
print("Starting model training...")
MAX_HORIZON = 48
models = {}

for h in range(1, MAX_HORIZON + 1):
    print(f"\n===== Training model for horizon t+{h} =====")

    # Use the TRANSFORMED 'out' column as the target
    y_train_h = train_df.groupby('Location')['out_transformed'].shift(-h)
    X_train_h = train_df[features]

    valid_indices = ~y_train_h.isna()
    X_train_h, y_train_h = X_train_h[valid_indices], y_train_h[valid_indices]

    # Create validation set for this horizon
    X_val_h = val_df[features]
    y_val_h = val_df.groupby('Location')['out_transformed'].shift(-h).dropna()
    X_val_h = X_val_h.loc[y_val_h.index]

    # ==> CHANGE 2: Objective is 'regression_l2' (RMSE)
    lgbm = lgb.LGBMRegressor(
        objective='regression_l2',
        n_estimators=3000,
        learning_rate=0.01,
        num_leaves=60,
        random_state=42,
        n_jobs=-1,
        colsample_bytree=0.7,
        subsample=0.6,
    )

    lgbm.fit(
        X_train_h, y_train_h,
        eval_set=[(X_val_h, y_val_h)],
        eval_metric='rmse',
        # ==> CHANGE 3: Callback will now print the loss as it improves
        callbacks=[lgb.early_stopping(100, verbose=True)]
    )

    models[h] = lgbm
    gc.collect()

print("\nModel training complete.")


# ===================================================================
# 5. Generate and Evaluate Predictions
# ===================================================================
print("Generating and evaluating predictions on the validation set...")

validation_predictions = []
for loc_id in val_df['Location'].unique():
    loc_df = val_df[val_df['Location'] == loc_id].copy()

    # Use the features from the first timestamp of the validation set for this location
    X_pred_loc = loc_df[features].iloc[[0]]

    for h in range(1, MAX_HORIZON + 1):
        if h > len(loc_df): continue # Cannot validate if horizon is longer than remaining data

        prediction_transformed = models[h].predict(X_pred_loc)[0]

        # ==> CHANGE 4: Reverse the transformation using expm1
        prediction_original_scale = np.expm1(prediction_transformed)
        prediction_original_scale = max(0, prediction_original_scale)

        # Get the true value from the original 'out' column
        true_value = loc_df['out'].iloc[h-1]

        validation_predictions.append({
            'pred': prediction_original_scale,
            'true': true_value
        })

final_preds_df = pd.DataFrame(validation_predictions)
rmse = np.sqrt(mean_squared_error(final_preds_df['true'], final_preds_df['pred']))
print(f"\n✅ Final Validation RMSE on original scale: {rmse:.4f}")